# This will train the model and also we create sample images of class 1

In [3]:
""" GAN-CLS """
import tensorflow as tf
import tensorlayer as tl
from tensorlayer.layers import *
from tensorlayer.prepro import *
from tensorlayer.cost import *
import numpy as np
import scipy
from scipy.io import loadmat
import time, os, re, nltk

from utils import *
from model import *
import model

In [4]:
print("Loading data from pickle ...")
import pickle
from sklearn.externals import joblib
with open("_vocab.sav", 'rb') as f:
    vocab = joblib.load(f)
print("done vocab load")
with open("_image_train.sav", 'rb') as f:
    _, images_train = joblib.load(f)
with open("_image_test.sav", 'rb') as f:
    _, images_test = joblib.load(f)
with open("_n.sav", 'rb') as f:
    n_captions_train, n_captions_test, n_captions_per_image, n_images_train, n_images_test = joblib.load(f)
with open("_caption.sav", 'rb') as f:
    captions_ids_train, captions_ids_test = joblib.load(f)
# images_train_256 = np.array(images_train_256)
# images_test_256 = np.array(images_test_256)
images_train = np.array(images_train)
images_test = np.array(images_test)

# print(n_captions_train, n_captions_test)
# exit()

ni = int(np.ceil(np.sqrt(batch_size)))
# os.system("mkdir samples")
# os.system("mkdir samples/step1_gan-cls")
# os.system("mkdir checkpoint")
tl.files.exists_or_mkdir("samples/step1_gan-cls")
tl.files.exists_or_mkdir("samples/step_pretrain_encoder")
tl.files.exists_or_mkdir("checkpoint")
save_dir = "checkpoint"

Loading data from pickle ...
done vocab load
[!] samples/step1_gan-cls exists ...
[!] samples/step_pretrain_encoder exists ...
[!] checkpoint exists ...


In [5]:
vocab

In [6]:
###======================== DEFIINE MODEL ===================================###
t_real_image = tf.placeholder('float32', [batch_size, image_size, image_size, 3], name = 'real_image')
t_wrong_image = tf.placeholder('float32', [batch_size ,image_size, image_size, 3], name = 'wrong_image')
t_real_caption = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name='real_caption_input')
t_wrong_caption = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name='wrong_caption_input')
t_z = tf.placeholder(tf.float32, [batch_size, z_dim], name='z_noise')

In [7]:
## training inference for text-to-image mapping
net_cnn = cnn_encoder(t_real_image, is_train=True, reuse=False)
x = net_cnn.outputs
v = rnn_embed(t_real_caption, is_train=True, reuse=False).outputs
x_w = cnn_encoder(t_wrong_image, is_train=True, reuse=True).outputs
v_w = rnn_embed(t_wrong_caption, is_train=True, reuse=True).outputs

alpha = 0.2 # margin alpha
rnn_loss = tf.reduce_mean(tf.maximum(0., alpha - cosine_similarity(x, v) + cosine_similarity(x, v_w))) + \
            tf.reduce_mean(tf.maximum(0., alpha - cosine_similarity(x, v) + cosine_similarity(x_w, v)))

  [TL] InputLayer  cnnftxt//in: (64, 64, 64, 3)
  [TL] Conv2dLayer cnnftxt/cnnf/h0/conv2d: shape:[4, 4, 3, 64] strides:[1, 2, 2, 1] pad:SAME act:<lambda>
  [TL] Conv2dLayer cnnftxt/cnnf/h1/conv2d: shape:[4, 4, 64, 128] strides:[1, 2, 2, 1] pad:SAME act:identity
  [TL] BatchNormLayer cnnftxt/cnnf/h1/batch_norm: decay:0.900000 epsilon:0.000010 act:<lambda> is_train:True
  [TL] Conv2dLayer cnnftxt/cnnf/h2/conv2d: shape:[4, 4, 128, 256] strides:[1, 2, 2, 1] pad:SAME act:identity
  [TL] BatchNormLayer cnnftxt/cnnf/h2/batch_norm: decay:0.900000 epsilon:0.000010 act:<lambda> is_train:True
  [TL] Conv2dLayer cnnftxt/cnnf/h3/conv2d: shape:[4, 4, 256, 512] strides:[1, 2, 2, 1] pad:SAME act:identity
  [TL] BatchNormLayer cnnftxt/cnnf/h3/batch_norm: decay:0.900000 epsilon:0.000010 act:<lambda> is_train:True
  [TL] FlattenLayer cnnftxt/cnnf/h4/flatten: 8192
  [TL] DenseLayer  cnnftxt/cnnf/h4/embed: 128 identity
  [TL] EmbeddingInputlayer rnnftxt/rnn/wordembed: (8000, 256)
  [TL] DynamicRNNLayer rnn

In [8]:
## training inference for txt2img
generator_txt2img = model.generator_txt2img_resnet
discriminator_txt2img = model.discriminator_txt2img_resnet

In [9]:
net_rnn = rnn_embed(t_real_caption, is_train=False, reuse=True)
net_fake_image, _ = generator_txt2img(t_z,
                net_rnn.outputs,
                is_train=True, reuse=False, batch_size=batch_size)
                #+ tf.random_normal(shape=net_rnn.outputs.get_shape(), mean=0, stddev=0.02), # NOISE ON RNN
net_d, disc_fake_image_logits = discriminator_txt2img(
                net_fake_image.outputs, net_rnn.outputs, is_train=True, reuse=False)
_, disc_real_image_logits = discriminator_txt2img(
                t_real_image, net_rnn.outputs, is_train=True, reuse=True)
_, disc_mismatch_logits = discriminator_txt2img(
                # t_wrong_image,
                t_real_image,
                # net_rnn.outputs,
                rnn_embed(t_wrong_caption, is_train=False, reuse=True).outputs,
                is_train=True, reuse=True)

## testing inference for txt2img
net_g, _ = generator_txt2img(t_z,
                rnn_embed(t_real_caption, is_train=False, reuse=True).outputs,
                is_train=False, reuse=True, batch_size=batch_size)

d_loss1 = tl.cost.sigmoid_cross_entropy(disc_real_image_logits, tf.ones_like(disc_real_image_logits), name='d1')
d_loss2 = tl.cost.sigmoid_cross_entropy(disc_mismatch_logits,  tf.zeros_like(disc_mismatch_logits), name='d2')
d_loss3 = tl.cost.sigmoid_cross_entropy(disc_fake_image_logits, tf.zeros_like(disc_fake_image_logits), name='d3')
d_loss = d_loss1 + (d_loss2 + d_loss3) * 0.5
g_loss = tl.cost.sigmoid_cross_entropy(disc_fake_image_logits, tf.ones_like(disc_fake_image_logits), name='g')

  [TL] EmbeddingInputlayer rnnftxt/rnn/wordembed: (8000, 256)
  [TL] DynamicRNNLayer rnnftxt/rnn/dynamic: n_hidden:128, in_dim:3 in_shape:(64, ?, 256) cell_fn:BasicLSTMCell dropout:None n_layer:1
       batch_size (concurrent processes): 64
  [TL] InputLayer  generator/g_inputz: (64, 512)
  [TL] InputLayer  generator/g_input_txt: (64, 128)
  [TL] DenseLayer  generator/g_reduce_text/dense: 128 <lambda>
  [TL] ConcatLayer generator/g_concat_z_txt: 640
  [TL] DenseLayer  generator/g_h0/dense: 16384 identity
  [TL] BatchNormLayer generator/g_h0/batch_norm: decay:0.900000 epsilon:0.000010 act:identity is_train:True
  [TL] ReshapeLayer generator/g_h0/reshape: (64, 4, 4, 1024)
  [TL] Conv2dLayer generator/g_h1_res/conv2d: shape:[1, 1, 1024, 256] strides:[1, 1, 1, 1] pad:VALID act:identity
  [TL] BatchNormLayer generator/g_h1_res/batch_norm: decay:0.900000 epsilon:0.000010 act:relu is_train:True
  [TL] Conv2dLayer generator/g_h1_res/conv2d2: shape:[3, 3, 256, 256] strides:[1, 1, 1, 1] pad:SAME

  [TL] Conv2dLayer discriminator/d_h3/conv2d: shape:[4, 4, 256, 512] strides:[1, 2, 2, 1] pad:SAME act:identity
  [TL] BatchNormLayer discriminator/d_h3/batchnorm: decay:0.900000 epsilon:0.000010 act:identity is_train:True
  [TL] Conv2dLayer discriminator/d_h4_res/conv2d: shape:[1, 1, 512, 128] strides:[1, 1, 1, 1] pad:VALID act:identity
  [TL] BatchNormLayer discriminator/d_h4_res/batchnorm: decay:0.900000 epsilon:0.000010 act:<lambda> is_train:True
  [TL] Conv2dLayer discriminator/d_h4_res/conv2d2: shape:[3, 3, 128, 128] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] BatchNormLayer discriminator/d_h4_res/batchnorm2: decay:0.900000 epsilon:0.000010 act:<lambda> is_train:True
  [TL] Conv2dLayer discriminator/d_h4_res/conv2d3: shape:[3, 3, 128, 512] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] BatchNormLayer discriminator/d_h4_res/batchnorm3: decay:0.900000 epsilon:0.000010 act:identity is_train:True
  [TL] ElementwiseLayer discriminator/d_h4/add: size:(64, 4, 4, 512) fn:add
  [

In [10]:
####======================== DEFINE TRAIN OPTS ==============================###
lr = 0.0002
lr_decay = 0.5      # decay factor for adam, https://github.com/reedscot/icml2016/blob/master/main_cls_int.lua  https://github.com/reedscot/icml2016/blob/master/scripts/train_flowers.sh
decay_every = 100   # https://github.com/reedscot/icml2016/blob/master/main_cls.lua
beta1 = 0.5

cnn_vars = tl.layers.get_variables_with_name('cnn', True, True)
rnn_vars = tl.layers.get_variables_with_name('rnn', True, True)
d_vars = tl.layers.get_variables_with_name('discriminator', True, True)
g_vars = tl.layers.get_variables_with_name('generator', True, True)

  [*] geting variables with cnn
  got   0: cnnftxt/cnnf/h0/conv2d/W_conv2d:0   (4, 4, 3, 64)
  got   1: cnnftxt/cnnf/h0/conv2d/b_conv2d:0   (64,)
  got   2: cnnftxt/cnnf/h1/conv2d/W_conv2d:0   (4, 4, 64, 128)
  got   3: cnnftxt/cnnf/h1/batch_norm/beta:0   (128,)
  got   4: cnnftxt/cnnf/h1/batch_norm/gamma:0   (128,)
  got   5: cnnftxt/cnnf/h2/conv2d/W_conv2d:0   (4, 4, 128, 256)
  got   6: cnnftxt/cnnf/h2/batch_norm/beta:0   (256,)
  got   7: cnnftxt/cnnf/h2/batch_norm/gamma:0   (256,)
  got   8: cnnftxt/cnnf/h3/conv2d/W_conv2d:0   (4, 4, 256, 512)
  got   9: cnnftxt/cnnf/h3/batch_norm/beta:0   (512,)
  got  10: cnnftxt/cnnf/h3/batch_norm/gamma:0   (512,)
  got  11: cnnftxt/cnnf/h4/embed/W:0   (8192, 128)
  [*] geting variables with rnn
  got   0: rnnftxt/rnn/wordembed/embeddings:0   (8000, 256)
  got   1: rnnftxt/rnn/dynamic/rnn/basic_lstm_cell/kernel:0   (384, 512)
  got   2: rnnftxt/rnn/dynamic/rnn/basic_lstm_cell/bias:0   (512,)
  [*] geting variables with discriminator
  got   0: 

In [12]:
with tf.variable_scope('learning_rate'):
    lr_v = tf.Variable(lr, trainable=False)
d_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(d_loss, var_list=d_vars )
g_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(g_loss, var_list=g_vars )
# e_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(e_loss, var_list=e_vars + c_vars)
grads, _ = tf.clip_by_global_norm(tf.gradients(rnn_loss, rnn_vars + cnn_vars), 10)
optimizer = tf.train.AdamOptimizer(lr_v, beta1=beta1)# optimizer = tf.train.GradientDescentOptimizer(lre)
rnn_optim = optimizer.apply_gradients(zip(grads, rnn_vars + cnn_vars))

C:\Users\cseuser\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [13]:
# adam_vars = tl.layers.get_variables_with_name('Adam', False, True)

###============================ TRAINING ====================================###
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
tl.layers.initialize_global_variables(sess)

# load the latest checkpoints
net_rnn_name = os.path.join(save_dir, 'net_rnn.npz')
net_cnn_name = os.path.join(save_dir, 'net_cnn.npz')
net_g_name = os.path.join(save_dir, 'net_g.npz')
net_d_name = os.path.join(save_dir, 'net_d.npz')

load_and_assign_npz(sess=sess, name=net_rnn_name, model=net_rnn)
load_and_assign_npz(sess=sess, name=net_cnn_name, model=net_cnn)
load_and_assign_npz(sess=sess, name=net_g_name, model=net_g)
load_and_assign_npz(sess=sess, name=net_d_name, model=net_d)

[*] Loading checkpoint\net_rnn.npz model SUCCESS!
[*] Loading checkpoint\net_cnn.npz model SUCCESS!
[*] Loading checkpoint\net_g.npz model SUCCESS!
[*] Loading checkpoint\net_d.npz model SUCCESS!


In [14]:
## seed for generation, z and sentence ids
## batch+size is 64 from models.py
sample_size = batch_size
sample_seed = np.random.normal(loc=0.0, scale=1.0, size=(sample_size, z_dim)).astype(np.float32)
    # sample_seed = np.random.uniform(low=-1, high=1, size=(sample_size, z_dim)).astype(np.float32)]
n = int(sample_size / ni)
sample_sentence = ["the flower shown has yellow anther red pistil and bright red petals."] * n + \
                  ["this flower has petals that are yellow, white and purple and has dark lines"] * n + \
                  ["the petals on this flower are white with a yellow center"] * n + \
                  ["this flower has a lot of small round pink petals."] * n + \
                  ["this flower is orange in color, and has petals that are ruffled and rounded."] * n + \
                  ["the flower has yellow petals and the center of it is brown."] * n + \
                  ["this flower has petals that are blue and white."] * n + \
                  ["this flower is orange and has pink dots"] * n

#["these white flowers have petals that start off white in color and end in a white towards the tips."] * n + \

In [15]:
test = ["this flower is orange and has pink dots"]

In [18]:
test[0] = [vocab.word_to_id(word) for word in nltk.tokenize.word_tokenize(sentence)] + [vocab.end_id]    # add END_ID

NameError: name 'sentence' is not defined

In [19]:
test = tl.prepro.pad_sequences(test, padding = "post")

ValueError: invalid literal for int() with base 10: 'this flower is orange and has pink dots'

In [20]:
# sample_sentence = captions_ids_test[0:sample_size]
# tokenize and add ids to the words. Also add a 2 (vocab.end_id) and some 0's from the prepro
for i, sentence in enumerate(sample_sentence):
    print("seed: %s" % sentence)
    sentence = preprocess_caption(sentence)
    sample_sentence[i] = [vocab.word_to_id(word) for word in nltk.tokenize.word_tokenize(sentence)] + [vocab.end_id]    # add END_ID
    # sample_sentence[i] = [vocab.word_to_id(word) for word in sentence]
    # print(sample_sentence[i])
sample_sentence = tl.prepro.pad_sequences(sample_sentence, padding='post')

seed: the flower shown has yellow anther red pistil and bright red petals.
seed: the flower shown has yellow anther red pistil and bright red petals.
seed: the flower shown has yellow anther red pistil and bright red petals.
seed: the flower shown has yellow anther red pistil and bright red petals.
seed: the flower shown has yellow anther red pistil and bright red petals.
seed: the flower shown has yellow anther red pistil and bright red petals.
seed: the flower shown has yellow anther red pistil and bright red petals.
seed: the flower shown has yellow anther red pistil and bright red petals.
seed: this flower has petals that are yellow, white and purple and has dark lines
seed: this flower has petals that are yellow, white and purple and has dark lines
seed: this flower has petals that are yellow, white and purple and has dark lines
seed: this flower has petals that are yellow, white and purple and has dark lines
seed: this flower has petals that are yellow, white and purple and has d

In [21]:
sample_sentence[60]

array([  6,   3,  17,  24,   5,   7,  16, 105,   2,   0,   0,   0,   0,
         0,   0])

# Training time

In [22]:
import os

In [23]:
cwd = os.getcwd()
img_dir = os.path.join(cwd, '102flowers')
class1_dir = os.path.join(cwd, 'text_c10\\class_00001')

In [24]:
alllines = []
for filename in os.listdir(class1_dir):
    
    if filename.endswith(".txt"):
        filepath = os.path.join(class1_dir, filename)
        text_file = open(filepath, "r")
        lines = text_file.read().split('\n')
        for a in lines:
            if a == "":
                continue
            alllines.append(a)

# This combines all the text for the class 1 images and saves it to alllines

In [28]:
from random import randrange

In [29]:
print(randrange(8))

2


# Below we run class1 text on the gan and save the output to class00001images

In [30]:
for i in range(0, len(alllines), 8):
    sample_size = batch_size
    sample_seed = np.random.normal(loc=0.0, scale=1.0, size=(sample_size, z_dim)).astype(np.float32)
        # sample_seed = np.random.uniform(low=-1, high=1, size=(sample_size, z_dim)).astype(np.float32)]
    n = int(sample_size / ni)
    sample_sentence = [alllines[i]] * n + \
                    [alllines[i+1]] * n + \
                    [alllines[i+2]] * n + \
                    [alllines[i+3]] * n + \
                    [alllines[i+4]] * n + \
                    [alllines[i+5]] * n + \
                    [alllines[i+6]] * n + \
                    [alllines[i+7]] * n 
    for j, sentence in enumerate(sample_sentence):
#         print("seed: %s" % sentence)
        sentence = preprocess_caption(sentence)
        sample_sentence[j] = [vocab.word_to_id(word) for word in nltk.tokenize.word_tokenize(sentence)] + [vocab.end_id]    # add END_ID
        # sample_sentence[i] = [vocab.word_to_id(word) for word in sentence]
        # print(sample_sentence[i])
    sample_sentence = tl.prepro.pad_sequences(sample_sentence, padding='post')   
    img_gen, rnn_out = sess.run([net_g.outputs, net_rnn.outputs], feed_dict={
                                    t_real_caption : sample_sentence,
                                    t_z : sample_seed})
    for index in range(0, 64, 8):
        imagetochoose = index + randrange(8)
        name = "class00001images/" + str(int(index / 8 + i)) + ".png"
        print("saving", name)
        scipy.misc.imsave(name, img_gen[imagetochoose])

saving class00001images/0.png
saving class00001images/1.png
saving class00001images/2.png
saving class00001images/3.png
saving class00001images/4.png
saving class00001images/5.png
saving class00001images/6.png
saving class00001images/7.png


C:\Users\cseuser\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


saving class00001images/8.png
saving class00001images/9.png
saving class00001images/10.png
saving class00001images/11.png
saving class00001images/12.png
saving class00001images/13.png
saving class00001images/14.png
saving class00001images/15.png
saving class00001images/16.png
saving class00001images/17.png
saving class00001images/18.png
saving class00001images/19.png
saving class00001images/20.png
saving class00001images/21.png
saving class00001images/22.png
saving class00001images/23.png
saving class00001images/24.png
saving class00001images/25.png
saving class00001images/26.png
saving class00001images/27.png
saving class00001images/28.png
saving class00001images/29.png
saving class00001images/30.png
saving class00001images/31.png
saving class00001images/32.png
saving class00001images/33.png
saving class00001images/34.png
saving class00001images/35.png
saving class00001images/36.png
saving class00001images/37.png
saving class00001images/38.png
saving class00001images/39.png
saving cla

saving class00001images/272.png
saving class00001images/273.png
saving class00001images/274.png
saving class00001images/275.png
saving class00001images/276.png
saving class00001images/277.png
saving class00001images/278.png
saving class00001images/279.png
saving class00001images/280.png
saving class00001images/281.png
saving class00001images/282.png
saving class00001images/283.png
saving class00001images/284.png
saving class00001images/285.png
saving class00001images/286.png
saving class00001images/287.png
saving class00001images/288.png
saving class00001images/289.png
saving class00001images/290.png
saving class00001images/291.png
saving class00001images/292.png
saving class00001images/293.png
saving class00001images/294.png
saving class00001images/295.png
saving class00001images/296.png
saving class00001images/297.png
saving class00001images/298.png
saving class00001images/299.png
saving class00001images/300.png
saving class00001images/301.png
saving class00001images/302.png
saving c

In [32]:
len(alllines)

400

In [33]:
img_gen, rnn_out = sess.run([net_g.outputs, net_rnn.outputs], feed_dict={
                                    t_real_caption : sample_sentence,
                                    t_z : sample_seed})

In [23]:
save_images(img_gen, [ni, ni], 'samples/step1_gan-cls/pinkflowerdottest.png')

In [24]:
img_gen[63].shape

(64, 64, 3)

In [25]:
# code to save an image
scipy.misc.imsave("class00001images\1.png", img_gen[63])

C:\Users\cseuser\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


# Will actually train it

In [ ]:
n_epoch = 10 # 600
print_freq = 1
n_batch_epoch = int(n_images_train / batch_size)
# exit()
for epoch in range(0, n_epoch+1):
    start_time = time.time()

    if epoch !=0 and (epoch % decay_every == 0):
        new_lr_decay = lr_decay ** (epoch // decay_every)
        sess.run(tf.assign(lr_v, lr * new_lr_decay))
        log = " ** new learning rate: %f" % (lr * new_lr_decay)
        print(log)
        # logging.debug(log)
    elif epoch == 0:
        log = " ** init lr: %f  decay_every_epoch: %d, lr_decay: %f" % (lr, decay_every, lr_decay)
        print(log)

    for step in range(n_batch_epoch):
        step_time = time.time()
        ## get matched text
        idexs = get_random_int(min=0, max=n_captions_train-1, number=batch_size)
        b_real_caption = captions_ids_train[idexs]
        b_real_caption = tl.prepro.pad_sequences(b_real_caption, padding='post')
        ## get real image
        b_real_images = images_train[np.floor(np.asarray(idexs).astype('float')/n_captions_per_image).astype('int')]
        # save_images(b_real_images, [ni, ni], 'samples/step1_gan-cls/train_00.png')
        ## get wrong caption
        idexs = get_random_int(min=0, max=n_captions_train-1, number=batch_size)
        b_wrong_caption = captions_ids_train[idexs]
        b_wrong_caption = tl.prepro.pad_sequences(b_wrong_caption, padding='post')
        ## get wrong image
        idexs2 = get_random_int(min=0, max=n_images_train-1, number=batch_size)
        b_wrong_images = images_train[idexs2]
        ## get noise
        b_z = np.random.normal(loc=0.0, scale=1.0, size=(sample_size, z_dim)).astype(np.float32)
            # b_z = np.random.uniform(low=-1, high=1, size=[batch_size, z_dim]).astype(np.float32)

        b_real_images = threading_data(b_real_images, prepro_img, mode='train')   # [0, 255] --> [-1, 1] + augmentation
        b_wrong_images = threading_data(b_wrong_images, prepro_img, mode='train')
        ## updates text-to-image mapping
        if epoch < 50:
            errRNN, _ = sess.run([rnn_loss, rnn_optim], feed_dict={
                                            t_real_image : b_real_images,
                                            t_wrong_image : b_wrong_images,
                                            t_real_caption : b_real_caption,
                                            t_wrong_caption : b_wrong_caption})
        else:
            errRNN = 0

        ## updates D
        errD, _ = sess.run([d_loss, d_optim], feed_dict={
                        t_real_image : b_real_images,
                        # t_wrong_image : b_wrong_images,
                        t_wrong_caption : b_wrong_caption,
                        t_real_caption : b_real_caption,
                        t_z : b_z})
        ## updates G
        errG, _ = sess.run([g_loss, g_optim], feed_dict={
                        t_real_caption : b_real_caption,
                        t_z : b_z})

        print("Epoch: [%2d/%2d] [%4d/%4d] time: %4.4fs, d_loss: %.8f, g_loss: %.8f, rnn_loss: %.8f" \
                    % (epoch, n_epoch, step, n_batch_epoch, time.time() - step_time, errD, errG, errRNN))

    if (epoch + 1) % print_freq == 0:
        print(" ** Epoch %d took %fs" % (epoch, time.time()-start_time))
        img_gen, rnn_out = sess.run([net_g.outputs, net_rnn.outputs], feed_dict={
                                    t_real_caption : sample_sentence,
                                    t_z : sample_seed})

        # img_gen = threading_data(img_gen, prepro_img, mode='rescale')
        save_images(img_gen, [ni, ni], 'samples/step1_gan-cls/train_{:02d}.png'.format(epoch))

    ## save model
    if (epoch != 0) and (epoch % 2) == 0:
        tl.files.save_npz(net_cnn.all_params, name=net_cnn_name, sess=sess)
        tl.files.save_npz(net_rnn.all_params, name=net_rnn_name, sess=sess)
        tl.files.save_npz(net_g.all_params, name=net_g_name, sess=sess)
        tl.files.save_npz(net_d.all_params, name=net_d_name, sess=sess)
        print("[*] Save checkpoints SUCCESS!")

    if (epoch != 0) and (epoch % 10) == 0:
        tl.files.save_npz(net_cnn.all_params, name=net_cnn_name+str(epoch), sess=sess)
        tl.files.save_npz(net_rnn.all_params, name=net_rnn_name+str(epoch), sess=sess)
        tl.files.save_npz(net_g.all_params, name=net_g_name+str(epoch), sess=sess)
        tl.files.save_npz(net_d.all_params, name=net_d_name+str(epoch), sess=sess)